In [2]:
from bokeh.models import ColumnDataSource, CategoricalColorMapper
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
import pandas as pd
output_notebook()

Loading BokehJS ...

In [112]:
# l2 = pd.read_csv("l2_res.csv")
# builtin = pd.read_csv("builtin_res.csv")
total = pd.read_csv("res.csv")
logb = pd.read_csv("log_res.csv")  

In [113]:
# mapper = {2:"BFGS", 5:"BFGS, restart=5", 6:"BFGS, restart=10"}
mapper = {2:"BFGS", 5:"ASD", 3:"SD", 4:"Built in"}
logmapper = {1:"BFGS_logb", 2:"BFGS_l2", 3:"SD", 4:"Built in", 5:"ASD"}
datas = {}
logdatas = {}
for i in mapper:
    datas[mapper[i]] = total[total['type'] == i]
for i in logmapper:
    logdatas[logmapper[i]] = logb[logb['type'] == i]
# datas['Built in'] = datas['Built in'][datas['Built in']['fmin'] < 0]

In [73]:
datas['Built in'][datas['Built in']['fmin'] > 0]

,type,size,x1,x2,fmin,time


In [70]:
vars = {}
for type in datas:
    vars[type] = datas[type].std()
vars

{'BFGS': type      0.000000
 size    288.949795
 x1        0.598407
 x2        0.287852
 fmin      0.275108
 time      2.155637
 dtype: float64,
 'Built in': type      0.000000
 size    288.949795
 x1        0.665250
 x2        0.331713
 fmin      0.282948
 time      0.118019
 dtype: float64,
 'Built in, restart=5': type      0.000000
 size    288.949795
 x1        0.671849
 x2        0.339088
 fmin      0.285094
 time      0.447098
 dtype: float64,
 'Built in, restart=10': type      0.000000
 size    288.949795
 x1        0.674137
 x2        0.332275
 fmin      0.283786
 time      0.944994
 dtype: float64}

In [114]:
bests = {}
avgs = {}
vars = {}
emas = {}
for type in datas:
    bests[type] = datas[type].groupby(['size']).min()
    avgs[type] = datas[type].groupby(['size']).mean()
    vars[type] = datas[type].groupby(['size']).std()
emas = pd.DataFrame({type:avgs[type].fmin for type in datas}).ewm(com=5).mean()
avgsLog = {}
for type in logdatas:
    avgsLog[type] = logdatas[type].groupby(['size']).mean()

In [97]:
# avgs["BFGS"][avgs["BFGS"].index > 700].mean()
avgs["Built in, restart=10"][avgs["Built in, restart=10"].index > 700].mean()

type    6.000000
x1     -0.006593
x2      0.642593
fmin   -0.166393
time    2.524735
dtype: float64

In [57]:
ratio_greater = avgs["BFGS"]/avgs["Built in"]

In [115]:
def visualise_pos(datas):
    p = figure(title = 'minimum f value ', plot_width = 900, x_axis_label = '|Y|', y_axis_label = 'minimum f value')
    colors = ['red', 'blue', 'green', 'black', 'orange']
    for type, col in zip(datas,colors):
        p.line(datas[type].fmin.index, abs(datas[type].fmin), color = col, legend_label = type)
    p.legend.location = 'top_right'
    show(p)

def visualise_time(datas):
    p = figure(title = 'avg run time', plot_width = 900, x_axis_label = '|Y|', y_axis_label = 'avg run time(s)')
    colors = ['red', 'blue', 'green', 'black', 'orange']
    for type, col in zip(datas,colors):
        p.line(datas[type].time.index, datas[type].time, color = col, legend_label = type)
    p.legend.location = 'top_left'
    show(p)

visualise_time(avgs)
visualise_pos(avgs)


In [91]:
def visualise_ema():
    p = figure(title = f'exponential moving average of average minimum f value', plot_width = 900, x_axis_label = 'size of |Y|', y_axis_label = 'ema', y_axis_type = 'log')
    for type, col in zip(emas.columns, ['red', 'blue', 'green', 'black']):
        p.line(emas[type].index, abs(emas[type]), color = col, legend_label = type)
    show(p)
visualise_ema()

In [60]:
def visualise_ratio():
    p = figure(title = f'ratio of BFGS:Built in', plot_width = 900, x_axis_label = 'size of |Y|', y_axis_label = 'ratio')
    p.line(ratio_greater.time.index, ratio_greater.time)
    show(p)
visualise_ratio()

In [15]:
l2_tries = pd.read_csv('l2_numTries_res.csv')
avgl2_NT = l2_tries.groupby(['size', 'numTries']).mean()
tries = {}
for i in range(1,5):
    tries[i] = l2_tries[l2_tries['numTries'] == i]
    tries[i] = tries[i].groupby('size').mean()
size = [1+10*i for i in range(len(tries[1]))]

def visualise_tries():
    p = figure(title = f'avg run time', plot_width = 900, x_axis_label = 'size of |Y|', y_axis_label = 'avergae run time (s)')
    colors = [' ', 'red', 'blue', 'green', 'black']
    for i in range(1,5):
        p.line(size, tries[i].fmin, color = colors[i], legend_label = str(i))
    show(p)

visualise_tries()

In [10]:
types = {}
for i in range(1,7):
    types[i] = l2[l2['type'] == i]
    types[i] = types[i].groupby('size').mean()
size = [1+10*i for i in range(len(types[1]))]

mapping = {1:"(k+1)^2", 5: "0.5(k+1)", 4:"k+1", 6:"2(k+1)", 2:"4(k+1)" }

def visualise_types():
    p = figure(title = f'avg run time', plot_width = 900, x_axis_label = 'size of |Y|', y_axis_label = 'avergae run time (s)')
    colors = ['red', 'blue', 'green', 'black', 'orange', 'purple', 'red']
    for i in mapping:
        p.line(size, types[i].time, color = colors[i], legend_label = mapping[i])
    show(p)

visualise_types()